In [1]:
import json
import pandas as pd
from unidecode import unidecode
import re
import math

with open('data/subset_epita.json') as f:
#with open('data/export_epita.json') as f:
    data = json.load(f)
    
def get_docs_from_id(id):
    docs = []
    for o in data:
        if o['document_type_id'] == id:
            docs.append(o)
    return docs

datas = get_docs_from_id(8)

def preprocess(word):
    lower = str.lower(word)
    normalized = unidecode(lower)
    punc = '''!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~'''
    for ele in normalized:
        if ele in punc:
            normalized = normalized.replace(ele, "")
    return normalized

def flatten(l):
    return [item for sublist in l for item in sublist]

# Récupérer les mots par paragraphe

In [2]:
def abs(a, b):
    r = a - b
    if r < 0:
        return -r
    return r

def regrouper_mots_par_hauteur(doc, tolerance):
    mots_regroupes = []
    
    keys = list(doc.keys())
    keys.pop()
    keys.pop()
    for key in keys:
        page = {}
        lines = doc[key]
        for line in lines:
            for w in line['line']['words']:
                hauteur = w['top']
                found = False
                for h in page:
                    if abs(h, hauteur) <= tolerance:
                        found = True
                        page[h].append(w['text'])
                        break
                if found == True:
                    continue
                if hauteur not in page:
                    page[hauteur] = []
                page[hauteur].append(w['text'])
        mots_regroupes.append(page)
    return mots_regroupes

tolerance = 0.02
docs = []

for doc in datas:
    resultat = regrouper_mots_par_hauteur(doc, tolerance)
    docs.append(resultat)

# Afficher les groupes de mots
doc = docs[29]
for page in doc:
    print("@@@@@@@@@@@@@@@@")
    for hauteur, mots in page.items():
        print(mots)

@@@@@@@@@@@@@@@@
['Laboratoire', 'BIOMAG', '-', 'BEAUMONT', 'BIOMAG', 'BEAUMONT', 'Autorisation', '950032482']
['1', 'rue', 'Louis', 'Blanc', '95260', 'BEAUMONT', 'SUR', 'OISE', 'Téléphone', ':', '01.30.28.68.20', 'Fax', ':', '01.39.37.78.13']
['labo.beaumont@biogroup.f', 'Site', 'internet', ':', 'www.biogroup.fr']
['BIOGROUP', 'biologie', 'médicale']
['Mme', 'Germain', 'Manon', 'Née', 'Germain', 'SAE', 'TANG', 'STEHANE']
['Date', 'de', 'naissance:', '22-08-1940', '(81', 'ans)', 'Sexe:', 'F', 'Dr', '25,', 'rue', 'de', '153', 'AVENUE', 'D', 'ITALIE']
['75013', '-', 'PARIS', 'CABINET', 'IPSO', 'ITALIE', 'Prescrit', 'par', 'le', 'Dr', 'SAE', 'TANG', 'STEHANE', 'CABINET', 'IPSO', 'ITALIE', '75013', 'PARIS', '13', '75013', 'PARIS', '13', '07:20']
['Sauf', 'mention', 'contraire,', 'prélevé', 'le', ':08-02-2022', 'Enregistré', 'le', '08-02-2022', '10:18']
['Edité', 'le', 'Jeudi', '10', 'Février', '2022', 'à', '14:50', '/', 'Compte-rendu', 'complet', 'Référence', '12202080083']
['Intervalle', 

# Récupérer le top des unités trouvées dans les documents

In [3]:
with open("utils/results/bioloinc_units.json", 'r') as fichier_json:
    units = json.load(fichier_json)

In [4]:
units

{'%  réactivité (avec DTT)': ['%'],
 '%  réactivité (sans DTT)': ['%'],
 '% CD10': ['%'],
 '% CD16+CD56+': ['%'],
 '% CD19+': ['%'],
 '% CD3': ['%'],
 '% CD3+CD4+CD8': ['%'],
 '% CD3-CD16+CD56+': ['%'],
 '% CD3-CD19+': ['%'],
 '% CD4': ['%'],
 '% CD45': ['%'],
 '% CD5': ['%'],
 '% CD8': ['%'],
 "% d'hématies foetales": ['%', '%{RBCs}'],
 "% d'hémoglobine foetale": ["% d'Hb", '%{Hb}'],
 '% de réactivité IgG anti-HLA classe 1': ['%'],
 '% de réactivité IgG anti-HLA classe 2': ['%'],
 '1,25 dihydroxy vitamines D': ['pmol/L', 'pg/mL'],
 '1-hydroxymidazolam': ['ng/mL'],
 '1-méthylhistidine': ['µmol/L', 'umol/L'],
 '1-méthylhistidine/créatinine': ['µmol/mmol créat', 'umol/mmol{creat}'],
 '11-déoxycortisol': ['nmol/L', 'ng/mL'],
 '11-déoxycortisol J01 après métopirone': ['nmol/L'],
 '11-déoxycortisol avant métopirone': ['nmol/L'],
 '17-hydroxyprogestérone': ['nmol/L', 'ng/mL'],
 '17-hydroxyprogestérone salivaire': ['nmol/L'],
 '17-hydroxyprégnénolone': ['nmol/L', 'ng/mL'],
 '2-aminoadipate': 

In [5]:
# Récupérer la liste des marqueurs avec le nombre d'occurences
top_120_df = pd.read_csv('utils/results/top_120.csv')

# Garder que les marqueurs qui sont apparus au moins une fois
top_50_df = top_120_df.drop('Unnamed: 0', axis=1)[:50]
top_50_list = list(map(lambda x : preprocess(x), list(top_50_df['grandeur'])))


# Modifier le dictionnaire des unités pour garder que ceux qui apparaissent
all_units = set()
units_found = {}
for key in units :
    normalized_key = preprocess(key)
    unit_lower = []
    for u in units[key]:
        all_units.add(u.lower())
        unit_lower += u.lower()
        
    if normalized_key in top_50_list:
        units_found[normalized_key] = unit_lower
        
all_units.add('t/l')
all_units.add('g/100ml')
all_units.add('/mm3')
all_units.add('giga/l')
all_units.add('u3')
all_units.add('/mm3')
all_units.add('picog.')
all_units.add('meg/l')
all_units.add('meq/l')
all_units.add('neg/ml')
all_units.add('10%/')
all_units.add('ng/nl')
all_units.remove('%')
len(all_units)

153

# Récupérer les synonymes des marqueurs

In [6]:
synonymes = pd.read_excel("data/variable_biologique_bioLOINC.xlsx", sheet_name="CKD1")

In [7]:
synonymes

,Variable (abréviation),Variable,Synonymes et exemples,RegExp,Unité (cible),"{RegExp unité, facteur d'échelle}",Plage valeur plausibles (unité cible),LOINC,Unité trouvé,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,ALB,Albuminurie volumique,"Albuminurie, Albumine (urines), Albumine urina...","[aA]lbuminurie|([aA]lbumin.{1,10}[uU]rin[^ ]*)...",mg/L,"{mg([ \/]|( par ))L, 1};{( g)([ \/]|( par ))L,...",0-10000,NaN,NaN,microalbuminurie : autre variable ?,NaN,NaN,NaN,NaN
1,ALB24h,Albuminurie 24h,"Albuminurie, Albumine (urines), Albumine urina...","[aA]lbuminurie|([aA]lbumin.{1,10}[uU]rin[^ ]*)...",mg/24h,"{mg([ \/]|( par ))24 ?h,1};{( g)([ \/]|( par )...",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CRE,Créatininurie volumique,"Créatininurie, Créatinine (urines), Créatinine...","[cC]r.atininurie|([[cR]r.atinin.{0,8}[uU]ri[^ ]*)",g/L,"{( g)([ \/]|( par ))L,1};{mg([ \/]|( par ))L, ...",0-10,NaN,umol/L,NaN,NaN,NaN,NaN,NaN
3,CRE24h,Créatininurie 24h,"Créatininurie, Créatinine (urines), Créatinine...","[cC]r.atininurie|([[cR]r.atinin.{0,8}[uU]ri[^ ]*)",g/24h,"{( g)([ \/]|( par ))24 ?h,1};{mg([ \/]|( par )...",0-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PRO,Protéinurie volumique,"Protéinuries, Protéines (urines), Protéines ur...","[pP]rot.inurie|([pP]rot.in.{1,10}[uU]rin[^ ]*)",mg/L,"{mg([ \/]|( par ))L, 1} ;{g([ \/]|( par ))L,1000}",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PRO24h,Protéinurie 24h,"Protéinuries, Protéines (urines), Protéines ur...","[pP]rot.inurie|([pP]rot.in.{1,10}[uU]rin[^ ]*)",mg/24h,"{mg([ \/]|( par ))24 ?h,1}; {g([ \/]|( par ))2...",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,RAC,Rapport albuminurie/créatininurie,"Rapport albuminurie/créatininurie, Rapport alb...","(RAC )|(ACR )|([rR]apport [micro]{0,5}?[aA]lbu...",mg/g,"{mg([ \/]|( par ))mmol,1};{g([ \/]|( par ))mol,1}",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,RPC,Rapport protéinurie/créatininurie,"Rapport protéines/créatinine urinaire, ...","(RPC )|([rR]apport [pP]rot.in.{1,10}[cC]r.atin...",mg/g,"{mg([ \/]|( par ))mmol,1};{g([ \/]|( par ))mol,1}",0-10000,NaN,NaN,utile?,NaN,NaN,NaN,NaN
8,DFG,Débit de filtration glomérulaire,"Débit de filtration glomérulaire, DFG, glomeru...","(DFG )|(GFR )|(CKD[- ]?EPI)|([dD].bit.{1,5}[fF...","mL/min/1,73m2","{m[Ll]\/min,1};{mL\/mi?n\/1.73m,1}",0-300,NaN,NaN,filtrer les formules ?,"{FRP34, 01/01/2023, ALB, 34, 5245-45}",NaN,NaN,"Regarder si mention de CKD-EPI ou MDRD, Exclur..."
9,HEMO,Hémoglobine,"Hb, Hémoglobine (attention a ne pas capter l'h...",\bH[bB]\b|\b[Hh]\.?émoglobine\b(?!(?:\s*[Aa]\d...,g/dL,"{g\/dL,1};{g\/100mL,1}",0-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
regex_syn = synonymes.iloc[:]['RegExp'].tolist()

list_regex_dict = []
for regex_unit in synonymes['{RegExp unité, facteur d\'échelle}']:
    unit_dict_list = []
    # Ex: {mg([ \/]|( par ))L, 1};{( g)([ \/]|( par ))L,0.001}
    for r_split in regex_unit.split(';'):
        # r_split ex: '{mg([ \/]|( par ))L, 1}'
        unit_dict = {}
        r_split_clean = r_split.replace('{', '').replace('}', '').strip()
            
        r_splits = r_split_clean.split(',')
        # ex r_splits : ['mg([ \/]|( par ))L', '1']
        
        unit_dict[r_splits[0]] = r_splits[1]
        unit_dict_list.append(unit_dict)
    list_regex_dict.append(unit_dict_list)

synonymes['Regex_unité_facteur'] = list_regex_dict
synonymes

,Variable (abréviation),Variable,Synonymes et exemples,RegExp,Unité (cible),"{RegExp unité, facteur d'échelle}",Plage valeur plausibles (unité cible),LOINC,Unité trouvé,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Regex_unité_facteur
0,ALB,Albuminurie volumique,"Albuminurie, Albumine (urines), Albumine urina...","[aA]lbuminurie|([aA]lbumin.{1,10}[uU]rin[^ ]*)...",mg/L,"{mg([ \/]|( par ))L, 1};{( g)([ \/]|( par ))L,...",0-10000,NaN,NaN,microalbuminurie : autre variable ?,NaN,NaN,NaN,NaN,"[{'mg([ \/]|( par ))L': ' 1'}, {'( g)([ \/]|( ..."
1,ALB24h,Albuminurie 24h,"Albuminurie, Albumine (urines), Albumine urina...","[aA]lbuminurie|([aA]lbumin.{1,10}[uU]rin[^ ]*)...",mg/24h,"{mg([ \/]|( par ))24 ?h,1};{( g)([ \/]|( par )...",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'mg([ \/]|( par ))24 ?h': '1'}, {'( g)([ \/]..."
2,CRE,Créatininurie volumique,"Créatininurie, Créatinine (urines), Créatinine...","[cC]r.atininurie|([[cR]r.atinin.{0,8}[uU]ri[^ ]*)",g/L,"{( g)([ \/]|( par ))L,1};{mg([ \/]|( par ))L, ...",0-10,NaN,umol/L,NaN,NaN,NaN,NaN,NaN,"[{'( g)([ \/]|( par ))L': '1'}, {'mg([ \/]|( p..."
3,CRE24h,Créatininurie 24h,"Créatininurie, Créatinine (urines), Créatinine...","[cC]r.atininurie|([[cR]r.atinin.{0,8}[uU]ri[^ ]*)",g/24h,"{( g)([ \/]|( par ))24 ?h,1};{mg([ \/]|( par )...",0-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'( g)([ \/]|( par ))24 ?h': '1'}, {'mg([ \/]..."
4,PRO,Protéinurie volumique,"Protéinuries, Protéines (urines), Protéines ur...","[pP]rot.inurie|([pP]rot.in.{1,10}[uU]rin[^ ]*)",mg/L,"{mg([ \/]|( par ))L, 1} ;{g([ \/]|( par ))L,1000}",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'mg([ \/]|( par ))L': ' 1'}, {'g([ \/]|( par..."
5,PRO24h,Protéinurie 24h,"Protéinuries, Protéines (urines), Protéines ur...","[pP]rot.inurie|([pP]rot.in.{1,10}[uU]rin[^ ]*)",mg/24h,"{mg([ \/]|( par ))24 ?h,1}; {g([ \/]|( par ))2...",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'mg([ \/]|( par ))24 ?h': '1'}, {'g([ \/]|( ..."
6,RAC,Rapport albuminurie/créatininurie,"Rapport albuminurie/créatininurie, Rapport alb...","(RAC )|(ACR )|([rR]apport [micro]{0,5}?[aA]lbu...",mg/g,"{mg([ \/]|( par ))mmol,1};{g([ \/]|( par ))mol,1}",0-10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'mg([ \/]|( par ))mmol': '1'}, {'g([ \/]|( p..."
7,RPC,Rapport protéinurie/créatininurie,"Rapport protéines/créatinine urinaire, ...","(RPC )|([rR]apport [pP]rot.in.{1,10}[cC]r.atin...",mg/g,"{mg([ \/]|( par ))mmol,1};{g([ \/]|( par ))mol,1}",0-10000,NaN,NaN,utile?,NaN,NaN,NaN,NaN,"[{'mg([ \/]|( par ))mmol': '1'}, {'g([ \/]|( p..."
8,DFG,Débit de filtration glomérulaire,"Débit de filtration glomérulaire, DFG, glomeru...","(DFG )|(GFR )|(CKD[- ]?EPI)|([dD].bit.{1,5}[fF...","mL/min/1,73m2","{m[Ll]\/min,1};{mL\/mi?n\/1.73m,1}",0-300,NaN,NaN,filtrer les formules ?,"{FRP34, 01/01/2023, ALB, 34, 5245-45}",NaN,NaN,"Regarder si mention de CKD-EPI ou MDRD, Exclur...","[{'m[Ll]\/min': '1'}, {'mL\/mi?n\/1.73m': '1'}]"
9,HEMO,Hémoglobine,"Hb, Hémoglobine (attention a ne pas capter l'h...",\bH[bB]\b|\b[Hh]\.?émoglobine\b(?!(?:\s*[Aa]\d...,g/dL,"{g\/dL,1};{g\/100mL,1}",0-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'g\/dL': '1'}, {'g\/100mL': '1'}]"


# Itérer sur les paragraphes pour trouver les marqueurs

In [14]:
def search_marqueur(words, start):
    line = ' '.join(words[start:])
    marqueur = None
    regex = None
    for reg in regex_syn:
        marqueur = re.search(r'(?i)'+reg, line)
        if marqueur:
            marqueur = marqueur.group()
            regex = reg
            break
            
    if not marqueur :
        return -1, None, None
    
    marqueur_splits = marqueur.split()
    index_marqueur = -1
    
    for i in range(len(words[start:])):
        if marqueur_splits[-1] in words[start:][i]:
            index_marqueur = i
            
    return index_marqueur, marqueur, regex

pattern = re.compile('^[<>]?[0-9]+[.,]?[0-9]*')

def search_unit(words, start, regex):
    regex_units = synonymes.loc[synonymes['RegExp'] == regex, 'Regex_unité_facteur'].values
    for i in range(start, len(words)):
        for j in range(len(regex_units)):
            for regex_unit in regex_units[j] :
                for (reg, facteur) in regex_unit.items():
                    unit = re.search(r'(?i)'+reg, words[i])
                    if unit:
                        unite_cible = synonymes.loc[(synonymes['RegExp'] == regex), 'Unité (cible)'].values[j]
                        return unite_cible, unit.group(), i, float(facteur), reg, j
    return None, None, -1, None, None, -1

def search_value(words, start, end, facteur):
    """
    Méthode : chercher à partir de l'unité la valeur (avant ou après)
    """
    index = end - 1
    while index >= start and re.search('^[<>]?[0-9]+[ .,]?[0-9]*$', words[index]) != None:
        index-=1
        
    value_str = ''.join(words[index+1:end]).replace(',','.').replace(' ', '')
    
    if value_str == '':
        index = end+1
        while index < len(words) and re.search('^[<>]?[0-9]+[ .,]?[0-9]*$', words[index]) != None:
            index+=1
        value_str = ''.join(words[end+1:index]).replace(',','.').replace(' ', '')
        if value_str == '':
            return None, None

    prefix = ''
    if not value_str[0].isdigit():
        prefix = value_str[0]
        value_float = float(value_str[1:])
    else :
        value_float = float(value_str)
    return prefix + str(float(f'{value_float * facteur:.2f}')), prefix + str(float(f'{value_float:.2f}'))

In [15]:
values_found_cible = {}
marqueurs_per_doc = []

def extract_values(doc):
    """
        doc : list of dict
    """
    values_found = {}
    for page in doc :
        l = []
        for words in page.values():
            # Etape 1 : trouver le marqueur
            index_marqueur, marqueur, marqueur_regex = search_marqueur(words, 0)
            while index_marqueur != -1 :
                # Etape 2 : trouver l'unité avec bioloinc
                unit_cible_in_doc, unit, index_unit, facteur, unit_regex, index_unit_regex = search_unit(words, index_marqueur, marqueur_regex)
                if index_unit == -1:
                    break
                        
                # Etape 3 : récupérer la valeur qui est entre le marqueur et l'unité
                value_cible, value = search_value(words, index_marqueur + 1, index_unit, facteur)
                if value == None:
                    break
                    
                # Etape 4 : stocker dans un dictionnaire
                values_found[preprocess(marqueur).strip()] = value_cible.strip() + unit_cible_in_doc.strip()
                values_found_cible[preprocess(marqueur).strip()] = value + unit + ' ; ' + value_cible + unit_cible_in_doc
                index_marqueur = synonymes.loc[(synonymes['RegExp'] == marqueur_regex), 'Regex_unité_facteur'].index.values[0]
                l.append(index_marqueur)
                
                # On relance pour vérifier s'il y a d'autres marqueurs sur la même ligne
                index_marqueur, marqueur, marqueur_regex = search_marqueur(words, index_unit)
                
        marqueurs_per_doc.append(l)

            
    return values_found

extract_values(docs[29])

{'leucocytes': '7.64G/L',
 'polynucleaires neutrophiles': '0.26G/L',
 'lymphocytes': '0.59G/L',
 'plaquettes': '293.0G/L',
 'hemoglobine a1c': '6.0%',
 'sodium': '143.0mmol/L',
 'potassium': '4.8mmol/L',
 'proteinurie': '90.0mg/L',
 'creatinine urinaire': '0.0g/L'}

In [16]:
for i in range(len(docs)) :
    print("Document index", i)
    print(extract_values(docs[i]))
    print('------------------')

Document index 0
{'leucocytes': '0.01G/L'}
------------------
Document index 1
{}
------------------
Document index 2
{}
------------------
Document index 3
{'debit de filtration glomerulaire': '122.42mL/min/1,73m2'}
------------------
Document index 4
{'hemoglobine': '13.5g/dL', 'leucocytes': '6.4G/L', 'lymphocytes': '2.52G/L', 'plaquettes': '264.0G/L', 'proteine c reactive': '<0.6mg/L', 'dfg': '95.0mL/min/1,73m2', 'sodium': '136.0mmol/L', 'potassium': '4.4mmol/L', 'ferritine': '22.0µg/L', 'asat': '20.0UI/L', 'sgpt': '11.0UI/L', 'phosphatases alcalines': '43.0UI/L', 'gammagt': '13.0UI/L', 'tsh': '0.98mUI/L'}
------------------
Document index 5
{'leucocytes': '5.1G/L', 'hemoglobine': '12.5g/dL', 'lymphocytes': '1.56G/L', 'plaquettes': '222.0G/L', 'ferritinex': '50.0µg/L'}
------------------
Document index 6
{}
------------------
Document index 7
{}
------------------
Document index 8
{}
------------------
Document index 9
{'leucocytes': '0.0G/L'}
------------------
Document index 10
{}

In [12]:
values_found_cible

{'leucocytes': '4.4Giga/L ; 4.4G/L',
 'polynucleaires neutrophiles': '2.3Giga/L ; 2.3G/L',
 'lymphocytes': '1.47Giga/L ; 1.47G/L',
 'plaquettes': '240.0Giga/L ; 240.0G/L',
 'hemoglobine a1c': '6.0% ; 6.0%',
 'sodium': '143.0mmol/L ; 143.0mmol/L',
 'potassium': '4.5mmol/L ; 4.5mmol/L',
 'proteinurie': '0.4g/L ; 400.0mg/L',
 'creatinine urinaire': '695.0mg/L ; 0.7g/L',
 'debit de filtration glomerulaire': '92.0ml/min ; 92.0mL/min/1,73m2',
 'hemoglobine': '13.2g/dL ; 13.2g/dL',
 'proteine c reactive': '0.3mg/L ; 0.3mg/L',
 'dfg': '116.0mL/mn/1.73m ; 116.0mL/min/1,73m2',
 'ferritine': '48.0ng/mL ; 48.0µg/L',
 'asat': '23.0U/L ; 23.0UI/L',
 'sgpt': '34.0UI/L ; 34.0UI/L',
 'phosphatases alcalines': '108.0U/L ; 108.0UI/L ',
 'gammagt': '17.0U/L ; 17.0UI/L',
 'tsh': '2.1mUI/L ; 2.1mUI/L',
 'ferritinex': '50.0ng/ml ; 50.0µg/L',
 'glycemie a jeun': '4.91mmol/L ; 4.91g/L',
 'alat': '23.0U/L ; 23.0UI/L',
 'gamma gt': '11.0U/L ; 11.0UI/L',
 'ggt': '15.0UI/L ; 15.0UI/L',
 'pn neutrophiles': '1927.0/

In [13]:
ma_liste = flatten(marqueurs_per_doc)
compteur = {}

for chiffre in ma_liste:
    marqueur = synonymes.loc[chiffre, 'Variable']
    if marqueur in compteur:
        compteur[marqueur] += 1
    else:
        compteur[marqueur] = 1
        
for var in synonymes['Variable']:
    if not var in compteur.keys():
        compteur[var] = 0
        
compteur

{'Leucocytes': 28,
 'Polynucléaires neutrophiles': 14,
 'Lymphocytes': 17,
 'Plaquettes': 18,
 'HbA1c': 2,
 'Sodium (sanguin)': 10,
 'Potassium (sanguin)': 10,
 'Protéinurie volumique': 3,
 'Créatininurie volumique': 3,
 'Débit de filtration glomérulaire': 13,
 'Hémoglobine': 17,
 'Protéine C réactive': 6,
 'Ferritine': 5,
 'ASAT,  Aspartate aminotransférase': 9,
 'ALAT, Alanine aminotransférase': 10,
 'PAL (phosphatase alcaline)': 7,
 'Gamma GT': 10,
 'TSH': 19,
 'Glycémie à jeun': 7,
 'Albuminurie volumique': 1,
 'B-HCG': 1,
 'Albuminurie 24h': 0,
 'Créatininurie 24h': 0,
 'Protéinurie 24h': 0,
 'Rapport albuminurie/créatininurie': 0,
 'Rapport protéinurie/créatininurie': 0,
 'Albumine (sanguine)': 0}